# Mechanical Turk HIT Grader

TODO: 
* Undo button Z
* Display approved images
* Make report function to see average time spent per image and total time spent 
* Detect Approved images copy and purge from original set and pool approved images
* If assignment status is already approved/rejected DO NOT INCLUDE IN GRADER
* Function to display by WorkerID

How the batch report approval process works:

Either Approve a submission by placing X in the column or provided rejection reason in the rejection column

I have streamlined this by using pyQT to create a UI that allows us to accept and reject by pressing "Y" or "N" which speeds up our efficiency to grade labels immensensely.

You can't really modify values in the existing CSV file. Instead, you need to:

- Import the batch report file as a data frame
- Modify the dataframe
- Export modified data frame to save results  

## Functions

In [1]:
def upscale(img):
#import cv2

    #img = cv2.imread('./test.bmp', cv2.IMREAD_UNCHANGED)

    #print('Original Dimensions : ',img.shape)

    scale_percent = 175 # percent of original size
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    # resize image
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

    #print('Resized Dimensions : ',resized.shape)

    #cv2.imshow("Resized image", resized)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    
    return resized

def displaybyWorkerID(batchName,workerID):
    
    ## create df
    ## find row values with workerID
    ## extract image_url at that row
    ## find image
    ## scan directory of annotated images to copy and move image_url + ".bmp"
    
    
    
    df = pd.read_csv("./Batch_Reports/" + batchName + "Report.csv")
    df = df.rename(columns={'Input.image_url': 'image_url'})
    
    ## extract image URL 
    ## get row number
    ## if row doesnt contain given workerID in workerID column skip to next image
    
    for image_path in glob.glob(".\\Batch_Pictures\\"+ batchName +"\\*.bmp"):

        image_url = Path(image_path).stem
        image_url = image_url.replace(".bmp",'')
        rownum = df.index[df.image_url == (image_url)]

        res = df.loc[rownum,'WorkerId']
        res = res.to_string(index = False)
        res = res.replace(" ","")
            
        if(res == workerID):
            #img = cv2.imread(image_path,0) #white line aesthetic
            img = cv2.imread(image_path,cv2.IMREAD_UNCHANGED)
            resized_img = upscale(img)
            cv2.imshow('image',resized_img)
            cv2.imwrite("./Batch_Pictures/" + workerID + "/" + image_url + ".bmp", resized_img)
            #rownum = 0

            k = cv2.waitKey(0)
            if k == 27:         # wait for ESC key to exit
                cv2.destroyAllWindows()
                break
            elif k == ord('y'): # wait for 'y' key
                cv2.destroyAllWindows()
                
    cv2.destroyAllWindows()
    return

def Grader(batchName):
    
    df = pd.read_csv("./Batch_Reports/" + batchName + "Report.csv")
    df = df.rename(columns={'Input.image_url': 'image_url'})
    
    for image_path in glob.glob(".\\Batch_Pictures\\"+ batchName +"\\*.bmp"):

        #img = cv2.imread(image_path,0) #white line aesthetic
        img = cv2.imread(image_path,cv2.IMREAD_UNCHANGED)
        resized_img = upscale(img)
        cv2.imshow('image',resized_img)

        #print(image_path)
        image_url = Path(image_path).stem
        image_url = image_url.replace(".bmp",'')
        #print(image_url)

        rownum = df.index[df.image_url == (image_url)]

        k = cv2.waitKey(0)
        if k == 27:         # wait for ESC key to exit
            cv2.destroyAllWindows()
            break
        elif k == ord('y'): # wait for 'y' key to approve
            df.at[rownum, 'Approve'] = 'x'
            cv2.destroyAllWindows()

            # Todo: MOVE IMAGE TO AN APPROVED FOLDER INSIDE NESTED BATCH FOLDER AND DELETE IN BATCH FOLDER

        elif k == ord('n'): # wait for 'n' key to reject
            df.at[rownum, 'Reject'] = 'Poor label: Not up to standard provided in video'
            cv2.destroyAllWindows()
            #update csv or list by entering key
            #update to corresponding row
            
    df.to_csv("./Batch_Reports/" + batchName + "Verdict.csv", index=False)
    
    return 
    

## Main Program

In [7]:
import glob
import cv2
import os
import pandas as pd
from pathlib import Path

In [8]:
# Batch Report Name
batchName = "Set4Batch"

In [ ]:
Grader(batchName)

## Review and extract results on a worker based on ID 
**I added this is in because I got email's from worker's claiming that they correctly labeled images when in fact did not.**

In [11]:
workerID = "A24H9XZ7OE8WMW"

os.mkdir("./Batch_Pictures/" + workerID)

for batchNumber in range (1,6):
    batchName = "Set" + str(batchNumber) + "Batch"
    displaybyWorkerID(batchName,workerID)